# Détecter et convertir des caractères encodés

Si la majorité des corpus aujourd’hui sont encodés en ASCII ou en UTF-8, il arrive que certains soient restés dans un encodage non standard (hors Unicode). Avant de se lancer dans leur analyse, une opération commune consiste à détecter le jeu de caractères utilisé lors de leur encodage pour le convertir dans un format plus exploitable.

## Détecter

Dans un monde idéal, le jeu de caractères d’un corpus est indiqué dans sa description. Dans la réalité, ce n’est pas toujours le cas ; pire, les données sont parfois encodées dans un format différent de celui déclaré.

Pour Python 3, il existe le module `chardet` (à installer avant utilisation) :

In [ ]:
import chardet

La méthode `.detect()` sert à détecter automatiquement l’encodage sur des données binaires :

In [ ]:
chardet.detect(b'Hello World!')

De la même manière, un fichier doit être ouvert en mode binaire pour lancer la détection :

In [ ]:
with open('./data/haiku.txt', 'rb') as haiku:
    text = haiku.read()
    print(chardet.detect(text))

La méthode `.chardet()` renvoie un dictionnaire, ce qui permet d’ouvrir facilement un fichier sans connaître au préalable son encodage :

In [ ]:
with open('./data/haiku.txt', 'rb') as haiku:
    text = haiku.read()
    result = chardet.detect(text)
    print(text.decode(result['encoding']))

## Convertir

Une fois le format détecté, le décodage en Python convertit automatiquement les caractères au format Unicode. La seule opération à réaliser est d’enregistrer le résultat dans un fichier :

In [ ]:
with open('./data/haiku.txt', 'rb') as haiku:
    text = haiku.read()
    result = chardet.detect(text)
    unicode = text.decode(result['encoding'])

with open('./data/haiku_utf8.txt', 'w') as dest:
    dest.write(unicode)

Il ne reste plus qu’à travailler avec le nouveau fichier, conforme aux standards !

## Un jeu d’espion

Nous savons détecter l’encodage de données binaires, comme nous savons décoder des données binaires encodées en ASCII. Mais comment procéder lorsque c’est le message lui-même qui contient les codes des caractères ?

Vous recevez une note accompagnée d’un message codé. La note indique juste `cp037` :

In [ ]:
with open('./data/secret-msg.txt') as secret:
    text = secret.read()

print(text)

Évidemment, tenter de décoder le message avec l’encodage indiqué dans la note ne sert à rien, car le message est bien en ASCII :

In [ ]:
import chardet

with open('./data/secret-msg.txt', 'rb') as secret:
    text = secret.read()

# naively trying to decode with given encoding
print(text.decode('cp037'))

# detection confirms the message is in ASCII
print(chardet.detect(text))

Le module `codecs` de Python peut ici nous venir en aide. En lisant le message avec un peu plus d’attention, on suspecte en effet être en présence de codes hexadécimaux, codes que peut aisément décoder la méthode `.decode()` du module :

In [ ]:
import codecs

with open('./data/secret-msg.txt') as secret:
    text = secret.read()

print(codecs.decode(text, 'hex'))

À notre disposition un objet de type `bytes` que la méthode native `.decode()` de Python peut facilement analyser :

In [ ]:
with open('./data/secret-msg.txt') as secret:
    text = secret.read()
    hexa = codecs.decode(text, 'hex')
    msg = hexa.decode('cp037')

print(msg)

Un meilleur espion aurait pu utiliser plusieurs encodages, voire dissimuler un message encodé dans un vrai message, le tout dans un fichier lui-même encodé différemment !